<a href="https://colab.research.google.com/github/masterlyj/self-LLM_Agent_RL/blob/main/4%20%E8%AF%84%E4%BC%B0LLMs/4_1_rouge_evaluations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 大语言模型项目

## 大语言模型策略的应用与实现

### 4.1 - BLEU, ROUGE 和 N-Grams

#### 使用 ROUGE 评估摘要质量

- **模型**: t5-base-cnn / t5-base
- **Colab 环境**: CPU
- **关键词**: 摘要评估, N-Grams, ROUGE

**相关背景**:

评估大语言模型（LLM）的方式与评估传统机器学习模型（如回归或分类）截然不同。在传统 ML 中，我们常用准确率（Accuracy）、F1 分数或召回率（Recall）。
生成式语言任务的指标是独特的。根据具体应用场景，我们会选择不同的指标来评估模型性能。
在本笔记本中，我们将探索使用 **ROUGE** 指标来衡量语言模型生成的摘要质量。

### 什么是 ROUGE?



ROUGE 并非单一指标，而是一组指标的集合，用于衡量**生成摘要**与作为基准的**参考摘要**之间的重叠度和相似性。

它通常返回四个单独的指标：

* **ROUGE-1**: 衡量一元组（Unigrams，即单个词或字）的重叠情况。
* **ROUGE-2**: 衡量二元组（Bigrams，即相邻双词）的重叠情况。
* **ROUGE-L**: 衡量最长公共子序列（Longest Common Subsequence），奖励生成摘要和参考摘要之间更长的连续一致序列。
* **ROUGE-LSUM**: 计算方式是将最长公共子序列的长度除以生成摘要和参考摘要长度之和。

### 我们要在这个项目中做什么？



我们将使用两个 T5 模型：一个是原始的 `t5-base` 模型，另一个是专门针对摘要生成任务微调过的 `t5-base-cnn`。

首先，我们将使用一个数据集，让两个模型分别生成摘要。通过比较生成的摘要，我们可以观察微调是否有效。换句话说，这一步我们只能确定两个模型的输出有显著差异，但还不知道谁更好。

为了确定哪个模型生成的摘要更好，我们将使用一个名为 `cnn_dailymail` 的知名数据集（可通过 `datasets` 库获取）。该数据集包含人工撰写的**参考摘要**用于对比。我们将把两个模型生成的摘要与这些参考摘要进行评估。

获得更高 ROUGE 分数的模型将被认为能产生更好的摘要。


### 涉及的模型

* **t5-Base 微调版**: `flax-community/t5-base-cnn-dm`
* **t5-Base 原版**: `t5-base`

---

### 1. 环境安装与配置

**中文适配说明**：为了演示中文评估，我们额外添加了 `jieba` 库用于中文分词。

In [ ]:
!pip install -q evaluate
!pip install -q transformers
!pip install -q rouge_score
!pip install -q kaggle
!pip install -q datasets
!pip install -q jieba

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import nltk
import jieba

nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
import torch
import os
import time

In [ ]:
### 加载数据

import os
import json
import zipfile
import pandas as pd
from google.colab import userdata

# 获取密钥信息
try:
    # 从 Secrets 获取用户名和 Key
    # 请确保你在 Secrets 里存的是 username 和 key
    username = userdata.get('KAGGLE_USERNAME')
    key = userdata.get('KAGGLE_KEY')

    # 创建配置目录
    kaggle_dir = os.path.expanduser('~/.kaggle')
    if not os.path.exists(kaggle_dir):
        os.makedirs(kaggle_dir)

    # 写入文件：构造 Kaggle CLI 能识别的标准格式
    kaggle_json_path = os.path.join(kaggle_dir, 'kaggle.json')
    with open(kaggle_json_path, 'w') as f:
        json.dump({"username": username, "key": key}, f)

except Exception as e:
    print("错误：请检查 Colab Secrets 是否分别设置了 KAGGLE_USERNAME 和 KAGGLE_KEY。")
    raise e

# 下载数据集
print("正在下载数据集...")
# 使用 --force 覆盖下载，防止断点续传导致的错误
!kaggle datasets download -d deepanshudalal09/mit-ai-news-published-till-2023 --force

# 解压文件
zip_file = "mit-ai-news-published-till-2023.zip"
extract_path = "./kaggle_data"

print("正在解压...")
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# 4: 读取并处理数据
csv_path = f"{extract_path}/articles.csv"

news = pd.read_csv(csv_path)

DOCUMENT = "Article Body"
MAX_NEWS = 3
articles = news.head(MAX_NEWS)[DOCUMENT].tolist()

print(f"\n数据加载完毕！成功读取 {len(articles)} 条新闻。")
print(f"第一条预览: {articles[0][:100]}...")

### 3. 加载模型并创建摘要

这两个模型都在 Hugging Face 上可用。

In [ ]:
model_name_base = "t5-base"
model_name_finetuned = "flax-community/t5-base-cnn-dm"

# 获取分词器和模型的辅助函数
def get_model(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    return tokenizer, model

# 加载两个模型
tokenizer_base, model_base = get_model(model_name_base)
tokenizer_finetuned, model_finetuned = get_model(model_name_finetuned)

定义生成摘要的函数：

In [ ]:
def create_summaries(texts_list, tokenizer, model, max_l=125):
    # 我们给每篇文章加个前缀，告诉 T5 模型要做什么任务
    prefix = "Summarize this news: "
    summaries_list = []

    texts_list = [prefix + text for text in texts_list]

    for text in texts_list:
        summary = ""
        # 计算编码
        input_encodings = tokenizer(text,
                       max_length=1024,
                       return_tensors='pt',
                       padding=True,
                       truncation=True
                  )
        # 生成摘要
        start = time.time()
        output = model.generate(
            input_ids=input_encodings.input_ids,
            attention_mask=input_encodings.attention_mask,
            max_length=max_l,  # 设置生成摘要的最大长度
            num_beams=2,       # 设置束搜索（beam search）的数量
            early_stopping=True
        )

        # 解码获取文本
        summary = tokenizer.batch_decode(output, skip_special_tokens=True)
        end = time.time()

        elapsed_time = end - start
        print(f"耗时: {elapsed_time:.3f} 秒")
        summaries_list += summary
    return summaries_list

分别使用两个模型生成摘要：

In [ ]:
print("--- 使用基础模型生成 ---")
summaries_base = create_summaries(articles, tokenizer_base, model_base)

print("\n--- 使用微调模型生成 ---")
summaries_finetuned = create_summaries(articles, tokenizer_finetuned, model_finetuned)

查看结果对比：

In [ ]:
print("基础模型摘要:", summaries_base)
print("微调模型摘要:", summaries_finetuned)

乍一看，摘要确实不同。但很难仅凭肉眼判断哪个更好。这就是我们需要 ROUGE 的原因。

---


### 4. ROUGE 评估

ROUGE 算法通常基于空格来切分单词（Tokens）。

#### 加载 ROUGE

In [ ]:
rouge_score = evaluate.load("rouge")

#### 定义计算函数 (适配中文)

下面的函数进行了修改，增加了 `is_chinese` 参数。如果是中文，会使用 `jieba` 进行分词并用空格连接，从而欺骗 ROUGE 算法使其能够正确处理中文。

In [ ]:
def compute_rouge_score(generated, reference, is_chinese=False):
    """
    计算 ROUGE 分数。
    :param generated: 生成的摘要列表
    :param reference: 参考摘要列表
    :param is_chinese: 是否为中文文本 (如果是，将启用 Jieba 分词)
    """

    # 预处理函数
    def process_text(texts, is_zh):
        processed = []
        for s in texts:
            s = s.strip()
            if is_zh:
                # 中文核心步骤：分词并用空格连接
                # "我爱AI" -> "我 爱 AI"
                tokens = jieba.cut(s)
                processed.append(" ".join(tokens))
            else:
                # 英文步骤：按句分割并换行 (Rouge 库的一般要求)
                processed.append("\n".join(sent_tokenize(s)))
        return processed

    generated_processed = process_text(generated, is_chinese)
    reference_processed = process_text(reference, is_chinese)

    # 英文通常可以使用 stemmer (词干提取)，中文不需要
    use_stemmer = not is_chinese

    return rouge_score.compute(
        predictions=generated_processed,
        references=reference_processed,
        use_stemmer=use_stemmer,
    )

#### 4.1 比较两个模型的输出 (英文环境)

这里我们比较 Base 模型和 Fine-tuned 模型的输出差异（注意：这里不是评估好坏，只是评估差异）。

In [ ]:
# 这里是英文，所以 is_chinese=False
scores = compute_rouge_score(summaries_base, summaries_finetuned, is_chinese=False)
print(scores)

### 结果分析：基础模型 vs 微调模型

我们对比了 **基础模型 (t5-base)** 和 **微调模型 (t5-base-cnn)** 在同一篇新闻上生成的摘要。计算得出的 ROUGE 分数如下：

* **ROUGE-1**: `0.47` (47%)
* **ROUGE-2**: `0.32` (32%)
* **ROUGE-L**: `0.34` (34%)

#### 详细解读

这一组数据揭示了微调（Fine-tuning）对模型产生的影响：**“核心内容一致，但表达风格迥异”。**

**1. ROUGE-1 (0.47) —— 关键词高度重叠**

* **含义**：衡量**单个词 (Unigrams)** 的重叠率。
* **分析**：接近 **50%** 的词汇是相同的。这说明两个模型都没有跑题，都准确抓住了原文中的核心实体（如 *"MIT"*, *"molecules"*, *"system"*）。微调并没有丢失文章的核心事实。

**2. ROUGE-2 (0.32) —— 遣词造句发生变化**

* **含义**：衡量**双词短语 (Bigrams)** 的重叠率（即相邻的两个词是否一样）。
* **分析**：分数相比 ROUGE-1 显著下降 (0.47 -> 0.32)。这意味着虽然大家用的“词”差不多，但**组合词的方式**变了。
* *例子*：基础模型可能说 *"predict molecular properties"*，而微调模型说 *"predict their properties"*。意思一样，但短语不同。



**3. ROUGE-L (0.34) —— 句式结构重构**

* **含义**：衡量**最长公共子序列**，反映句子结构的相似度。
* **分析**：**34%** 的分数表明两个模型的**句法结构**有明显差异。
* 基础模型倾向于机械地拼接原文句子（陈述句）。
* 微调模型学会了新闻报道的风格（例如使用 *"Researchers created..."* 作为主语），导致句子结构与基础模型不再对齐。



> **结论**  
> 这组数据证明微调是**有效**的：模型不仅仅是在复制基础模型的能力，它成功习得了新的**语言风格（News Style）**，同时保留了**核心信息准确性（High ROUGE-1）**。

### 5. 与真实数据集对比 (评估质量)

为了判断谁更好，我们需要与“标准答案”对比。我们将加载 `cnn_dailymail` 数据集。

In [ ]:
from datasets import load_dataset

print("正在加载 CNN/DailyMail 数据集...")
cnn_dataset = load_dataset("cnn_dailymail", "3.0.0")

print("正在提取测试数据...")
sample_cnn = cnn_dataset["test"].select(range(MAX_NEWS))

real_articles = sample_cnn["article"]
real_summaries = sample_cnn["highlights"]

print(f"成功加载测试数据！共 {len(real_articles)} 条。")
print(f"第一条参考摘要预览: {real_summaries[0][:100]}...")

让两个模型针对这些新闻生成摘要：

In [ ]:
# 获取参考摘要的最大长度作为约束
max_length = max(len(item) for item in real_summaries) + 10

summaries_t5_base = create_summaries(real_articles, tokenizer_base, model_base, max_l=max_length)
summaries_t5_finetuned = create_summaries(real_articles, tokenizer_finetuned, model_finetuned, max_l=max_length)

In [ ]:
summaries = pd.DataFrame.from_dict(
        {
            "base": summaries_t5_base,
            "finetuned": summaries_t5_finetuned,
            "reference": real_summaries,
        }
    )
summaries.head()

#### 计算 ROUGE 分数

In [ ]:
print("--- 基础模型 (Base) 真实得分 ---")
# 预测值：基础模型生成的摘要
# 参考值：人工写的 Highlights
score_base = compute_rouge_score(summaries_t5_base, real_summaries, is_chinese=False)
print(score_base)

# 3. 计算 微调模型 vs 标准答案
print("\n--- 微调模型 (Fine-tuned) 真实得分 ---")
# 预测值：微调模型生成的摘要
# 参考值：人工写的 Highlights
score_finetuned = compute_rouge_score(summaries_t5_finetuned, real_summaries, is_chinese=False)
print(score_finetuned)

测了好几遍结果，基础模型的总结摘要比微调模型会更好一点，不理解哪里出问题了

In [ ]:
entities=['Paris, Londres, Barcelona, Reus']
entities_ref=['Reus, Paris, Londres, Barcelona']

In [ ]:
compute_rouge_score(entities, entities_ref)

In [ ]:
entities_ref=['Paris, Londres, Barcelona, Reus']
compute_rouge_score(entities, entities_ref)